This is simple explanation for calculating the price of an European call option for a non dividend paying stock using both Black-Scholes Merton and Monte Carlo Simulation method. The payoff of the call option is 𝑪 = 𝐦𝐚𝐱{𝑺 − 𝑲, 𝟎} where 𝑺 is the stock price at the end of Time.

Assuming continuous frequency (= daily simulation). Assuming the following model parameters: 𝑆0 = 100 , 𝑟 = 5%, 𝑇 = 1 𝑦𝑒𝑎𝑟, 𝐾 = 100, 𝜎 = 20%.
Greeks are also computed using both Black-Scholes Merton and Monte Carlo Simulation method

Theory for generating Monte Carlo Paths - Stock Prices are assumed to follow Geometric Brownian Motion (GBM). Research the Internet or John C Hull's book on Derivatives for more

Theory for generating Black - Scholes Merton Price - It is a closed form solution. Research the Internet John C Hull's book on Derivatives for more

In [144]:
# Import Numpy, Pandas and norm from Scipy stats
# One may also import seaborn if any charts are needed for graphical representation of the paths taken in Monte Carlo Simul
import numpy as np
import pandas as pd
from scipy.stats import norm

N_prime = norm.pdf
N = norm.cdf

In [145]:
# A Monte Carlo Simulation Function

def MCS(S0=100, rf=0.05, T=1, K=100, vol=0.2, steps=252, sims=10000): 
    # Steps are equal to days in a time period to expiry. It can be smaller or larger as per user
    # dt in this case is 1 day. This is the time interval for each steps in the simulation
    dt = T/steps 
    # Creating a zero filled array of size = sims x steps + 1. This will be simulation of price paths taken
    S = np.zeros((sims, steps + 1)) 
    # The first price for each path is S0
    S[:,0] = S0
    # All the steps for all the paths can now be filled using a 'for' loop implementing GBM
    for i in range (steps):
        # phi consists of an 1D array of standard normal random numbers of length sims
        phi = np.random.randn(sims)
        # using GBM formula, create path wise prices. phi is the proxy for Z
        S[:, i+1] = S[:, i ] * np.exp((rf - 0.5 * vol * vol) * dt + vol * phi * np.sqrt(dt))
    S = pd.DataFrame(S)
    # Payoff is the average of (S - K)+
    S['payoff'] = np.maximum(S[[steps]] - K,0)
    payoff = S['payoff'].mean(axis = 0)
    # Price of the Call Option is the expected discounted payoff
    C_MCS = np.exp(-rf * T)*payoff    
    return C_MCS       
        

In [146]:
# A Black Scholes Pricing Function and Greeks

def BSM(S0=100, rf=0.05, T=1, K=100, vol=0.2):
    # Formula for d1
    d1 = (np.log(S0/K) + (rf + 0.5 * vol * vol) * T)/(vol * np.sqrt(T))
    # Formula for d2
    d2 = d1 - vol * np.sqrt(T)
    # Formula for European Call
    price = S0 * N(d1) - K * np.exp(-rf * T) * N(d2)
    # Formula for Delta
    delta = N(d1)
    # Formula for Gamma
    gamma = N_prime(d1) / (S0 * vol * np.sqrt(T))
    # Formula for vega
    vega = N_prime(d1) * S0 * np.sqrt(T)
    # Formula for rho
    rho = K * T * np.exp(-rf * T) *N(d2)
    # Formula for theta
    theta = -S0 * N_prime(d1) * vol / (2 * np.sqrt(T)) - rf * K * np.exp(-rf * T) * N(d2)
    return [price,delta,gamma,vega,rho,theta]

In [147]:
# Function Calling

# Parameters
S0 = 100
rf = 0.05
T = 1
K = 100
vol = 0.20
steps = 252
sims = 10000

np.random.seed(40) 

# output array of price S at time T loaded to a dataframe
C_MCS = MCS(S0, rf, T, K, vol, steps, sims)

print("MonteCarlo call price is ",np.round(C_MCS,2))

# BSM output price
BSM_Output = BSM(S0, rf, T, K, vol)
print("BSM Vanilla call price is ",np.round(BSM_Output[0],2))


MonteCarlo call price is  10.74
BSM Vanilla call price is  10.45


In [148]:
# Greeks
# Delta

# Monte Carlo
# Checking for 1 dollar change in price
DelS = 1

np.random.seed(42) 
# Call price by increasing the asset price by DelS
C_MCS_Up = MCS(S0+DelS, rf, T, K, vol, steps, sims)
# Call price by decreasing the asset price by DelS
C_MCS_Down = MCS(S0-DelS, rf, T, K, vol, steps, sims)
# Center Delta computation
Delta = (C_MCS_Up - C_MCS_Down)/2
print("The MCS Delta is " , np.round(Delta,4))
# BSM Delta
print("The BSM Delta is ",np.round(BSM_Output[1],4))

The MCS Delta is  0.6259
The BSM Delta is  0.6368


In [149]:
# Greeks
# Gamma

# Monte Carlo
DelS = 2

np.random.seed(42) 
# Call price by increasing the asset price by DelS
C_MCS_Up = MCS(S0+DelS, rf, T, K, vol, steps, sims)
# Call price by decreasing the asset price by DelS
C_MCS_Down = MCS(S0-DelS, rf, T, K, vol, steps, sims)
# Center change in call price computation
DelC = (C_MCS_Up - C_MCS_Down)/2
# Center Gamma computation
Gamma = ((DelC - Delta * DelS)*2)
print("The MCS Gamma is " , np.round(Gamma,4))
# BSM Gamma
print("The BSM Gamma is ",np.round(BSM_Output[2],4))

The MCS Gamma is  0.0196
The BSM Gamma is  0.0188


In [150]:
# Greeks
# Vega

# Monte Carlo
# Checking for 100% change in vol
DelVol = 1

np.random.seed(42) 

# Call price by increasing the vol by DelVol
C_MCS_Up = MCS(S0, rf, T, K, vol+DelVol, steps, sims)
# Vega computation
Vega = (C_MCS_Up - C_MCS)
print("The MCS Vega is " , np.round(Vega,4))
# BSM Vega
print("The BSM Vega is ",np.round(BSM_Output[3],4))

The MCS Vega is  33.7715
The BSM Vega is  37.524


In [151]:
# Greeks
# Rho

# Monte Carlo
# Checking for 1% change in rf
Delrf = 1

np.random.seed(42) 
# Call price by increasing the rf by Delrf
C_MCS_Up = MCS(S0, rf+Delrf, T, K, vol, steps, sims)
# Call price by decreasing the rf by Delrf
C_MCS_Down = MCS(S0-DelS, rf-Delrf, T, K, vol, steps, sims)
# Center Rho computation
Rho = (C_MCS_Up - C_MCS_Down)/2
print("The MCS Rho is " , np.round(Rho,4))
# BSM Rho
print("The BSM Rho is ",np.round(BSM_Output[4],4))

The MCS Rho is  32.4246
The BSM Rho is  53.2325


In [153]:
# Greeks
# Theta

# Monte Carlo
# Checking for 1 year change in T
DelT = 1

np.random.seed(42) 
# Call price by increasing the T by DelT
C_MCS_Up = MCS(S0, rf, T+DelT, K, vol, steps, sims)
# Theta computation
Theta = (C_MCS_Up - C_MCS) * -1
print("The MCS Theta is " , np.round(Theta,4))
# BSM Theta
print("The BSM Theta is ",np.round(BSM_Output[5],4))

The MCS Theta is  -5.2213
The BSM Theta is  -6.414
